## Откройте и изучите файл

In [2]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

df = pd.read_csv('/datasets/users_behavior.csv')

print(df.shape)
print(df.head(5)) 

(3214, 5)
   calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0


Вывод: 1. Предобработка данных не требуется. 2. Столбцы calls, minutes,massages,mb_used - возьмем за признаки. 3 Столбец - is_ultra целевой признак, модель должна будет предсказать 0 или 1, 1 - тариф ультра, 0 - тариф смарт.

## Разбейте данные на выборки

In [3]:
#разбиваю данны на тренировочную, валидационную и тестовую выборки в соотношении - 60%, 20%, 20%

df_train, df_valid = train_test_split(df, test_size=0.4, random_state=12345) 

df_valid, df_test = train_test_split(df_valid, test_size=0.5, random_state=12345)


print(df_train.shape)
print(df_valid.shape)
print(df_test.shape)

(1928, 5)
(643, 5)
(643, 5)


## Исследуйте модели

In [4]:
#разделение данных на признаки и целевой признак


features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']


In [5]:
#Подбор лучших гиперпараметров для Случайного леса

parametrs = { 'n_estimators': range (10, 51, 10),
              'max_depth': range (1,13, 2),
              'min_samples_leaf': range (1,8),
              'min_samples_split': range (2,10,2) }


clf = RandomForestClassifier()
grid = GridSearchCV(clf, parametrs, cv=5)
grid.fit(features_train, target_train)

print(grid.best_params_)

{'max_depth': 11, 'min_samples_leaf': 3, 'min_samples_split': 6, 'n_estimators': 50}


In [6]:
#Модель случайный лес

model = RandomForestClassifier(max_depth = 9,min_samples_leaf=5,min_samples_split=2,n_estimators=40)
model.fit(features_train,target_train) 
result = model.score(features_valid,target_valid) 
    
print("Accuracy модели случайного леса на валидационной выборке:", result)

Accuracy модели случайного леса на валидационной выборке: 0.7947122861586314


In [7]:
#Подбор лучших гиперпараметров для Дерева решений

parametrs = { 'max_depth': range (1,13, 2),
              'min_samples_leaf': range (1,8),
              'min_samples_split': range (2,10,2) }


clf = DecisionTreeClassifier()
grid = GridSearchCV(clf, parametrs, cv=5)
grid.fit(features_train, target_train)

print(grid.best_params_)

{'max_depth': 7, 'min_samples_leaf': 7, 'min_samples_split': 4}


In [8]:
#Дерево решений

model = DecisionTreeClassifier(max_depth=5,min_samples_leaf=1,min_samples_split=2)
model.fit(features_train, target_train)
predictions_valid=model.predict(features_valid)
print("Accuracy модели дерева решений на валидационной выборке:",accuracy_score(target_valid, predictions_valid)) 

Accuracy модели дерева решений на валидационной выборке: 0.7791601866251944


## Проверьте модель на тестовой выборке

In [61]:
model = RandomForestClassifier(max_depth = 9,min_samples_leaf=5,min_samples_split=2,n_estimators=40)
model.fit(features_train,target_train) 
result = model.score(features_test,target_test) 

print("Accuracy модели случайного леса на тестовой выборке:", result)


Accuracy модели на тестовой выборке: 0.8009331259720062


In [63]:
model = DecisionTreeClassifier(max_depth=5,min_samples_leaf=1,min_samples_split=2)
model.fit(features_train,target_train) 
result = model.score(features_test,target_test) 

print("Accuracy модели дерева решений на тестовой выборке:", result)

Accuracy модели дерева решений на тестовой выборке: 0.7822706065318819


In [64]:
results=pd.DataFrame({'Модель/Accuracy ':['Случайный лес','Дерево решений'],
                      'Валидационная выборка': ['0.79', '0.77'],
                      'Тестовая выборка': ['0.80', '0.78'],
                         }) 

print(results)

  Модель/Accuracy  Валидационная выборка Тестовая выборка
0    Случайный лес                  0.79             0.80
1   Дерево решений                  0.77             0.78


## (бонус) Проверьте модели на адекватность

In [41]:

model = RandomForestClassifier(max_depth = 9,min_samples_leaf=5,min_samples_split=2,n_estimators=40)
model.fit(features_train,target_train) 
predictions_test = model.predict(features_test)

result =mean_squared_error(target_test, predictions_test)**0.5
print("RMSE модели случайного леса на тестовой выборке:", result)

RMSE модели линейной регрессии на валидационной выборке: 0.45136737131523463


In [42]:
model = DecisionTreeClassifier(max_depth=5,min_samples_leaf=1,min_samples_split=2)
model.fit(features_train, target_train)
predictions_test = model.predict(features_test)

result =mean_squared_error(target_test, predictions_test)**0.5
print("RMSE модели дерева решений на тестовой выборке:", result)

RMSE модели дерева решений на тестовой выборке: 0.4649453554072505


In [67]:
from sklearn.dummy import DummyClassifier
dummy_classifier = DummyClassifier(strategy="most_frequent")
dummy_classifier.fit( features_train,target_train)
result = dummy_classifier.score(features_test,target_test) 

print("Accuracy случайной модели:", result)

Accuracy случайной модели: 0.6842923794712286


## 6. Вывод

1.Для обучения модели разбила данные на выборки - тренировочную, валидационную и тестовую - в соотношении 60%, 20%, 20%.
2.С помощью GridSearchCV() подобрала лучшие гиперпараметры для моделей.
3.Обучила модель Случайного леса и  Дерева решений, посчитала значение accuracy на валидационной выборке. Лучшее значение у модели  случайного леса = 0.79.
4.Проверила модель на тестовой выборе, acuuracy = 0.8
5.Проверила модели на адекватность, для этого рассчитала параметр RMSE. В двух случаях он равен 0,45 и 0,46. Значение низкое, модель адекватна.